In [1]:

%matplotlib inline
from __future__ import unicode_literals, print_function, division

import unicodedata
import string
import re
import time
import os
import glob
import csv
import copy

from pathlib import Path
import pandas as pd
import math
import random
import pickle
import gc
import pretty_midi
from midiutil.MidiFile import MIDIFile
from music21 import *
from io import open
import numpy as np
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from itertools import groupby
import librosa

from midi2audio import FluidSynth

import scipy

import datetime
from pathlib import Path
import math
matplotlib.use('Agg')
# matplotlib.use("QtAgg")
import ffmpeg
#conda install -c conda-forge ffmpeg-python

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
plt.rcParams['animation.ffmpeg_path'] = '/home/ilc/anaconda3/bin/ffmpeg'#'/usr/local/bin/ffmpeg'

import numpy as np
import subprocess as sp
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

import pretty_midi
import numpy as np
import pickle

import torch
import torch.nn as nn
from torch import Tensor
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
# from torch.nn import TransformerEncoder, TransformerEncoderLayer
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
change_fps = 40 #40

In [46]:
this_performer = "vio01"
performaer_list = ["vio01"] #, "vio02", "vio03", "vio04", "vio05"
test_pieces = ['Elgar_S1_T1', 'Flower_S1_T1']
test_pieces_eval = ['Elgar_S1_T1', 'Elgar_S1_T2',
                   'Flower_S1_T1', 'Flower_S1_T2',
                   'Mend_S1_T1', 'Mend_S1_T2',
                   'Mozart1_S1_T1', 'Mozart1_S1_T2',
                   'Mozart2_S1_T1', 'Mozart2_S1_T2']
test_pieces_test = ['vs1-1ada'] #"vs1-2fug", 
# piece_list = ["Bach1_S1_T1", "Bach1_S1_T2", "Bach2_S1_T1", "Bach2_S1_T2", "Bach2_S2_T1", "Bach2_S2_T2", "Beeth1_S1_T1", "Beeth1_S1_T2", "Beeth2_S1_T1", "Beeth2_S1_T2", "Elgar_S1_T1", "Elgar_S1_T2", "Flower_S1_T1", "Flower_S1_T2", "Mend_S1_T1", "Mend_S1_T2", "Mozart1_S1_T1", "Mozart1_S1_T2", "Mozart2_S1_T1", "Mozart2_S1_T2", "Wind_S1_T1", "Wind_S1_T2"]

In [4]:
# UoE_mocap_path = "./UoE_violin_midi/performance_motion/"

In [5]:
def read_midi(filename):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(filename)

    # Get the piano roll representation of the MIDI file
    # fs = 40
    piano_roll = midi_data.get_piano_roll(fs=change_fps) #40fps #250fps
    piano_roll[piano_roll > 0] = 1

    return piano_roll

def audio_preprocess(audio_path, specific_fps):
    n_fft = 4096
    hop = int(44000/specific_fps)  # 1102.5 -> 40fps #882 -> 50fps
    y, sr = librosa.load(audio_path, sr=44000)  # 44000 for divide 40
    print("y.shape", y.shape)
    print("sample rate: ", sr)
    mfcc = librosa.feature.mfcc(
        y=y, sr=sr, n_fft=n_fft, hop_length=hop, n_mfcc=13)
    y = np.where(y == 0, 1e-10, y)
    energy = np.log(librosa.feature.rms(
        y=y, frame_length=n_fft, hop_length=hop, center=True))
    mfcc_energy = np.vstack((mfcc, energy))
    mfcc_delta = librosa.feature.delta(mfcc_energy)

    sgram = librosa.stft(y, n_fft=n_fft, hop_length=hop)
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(
        S=sgram_mag, sr=sr)

    print("mfcc_energy", mfcc_energy.shape)
    print("mfcc_delta", mfcc_delta.shape)
    print("mel_scale_sgram", mel_scale_sgram.shape)

    aud = np.vstack((mfcc_energy, mfcc_delta, mel_scale_sgram)).T

    print("hop:", hop)
    print("aud:", aud.shape)
    return aud

def read_motion(motion_path):
    file = open(motion_path, 'rb')
    motion_data = pickle.load(file)
    file.close()

    return motion_data
    

In [6]:
UoE_violin_midi_path = "./UoE_violin_midi/violin_midi/vio01/"
UoE_violin_audio_path = "./UoE_violin_midi/performance_audio/vio01/"
UoE_violin_motion_path = "./preprocessed_data_save_cross/motion/"
#midi_path_list = glob.glob(UoE_violin_midi_path + "/" + this_performer + "/*.mid")

piano_roll_dictionary = {}
piano_roll_dictionary[this_performer] = {}
piano_roll_length_list = []

audio_dictionary = {}
audio_dictionary[this_performer] = {}
audio_len_list = []

motion_dictionary = {}
motion_dictionary[this_performer] = {}
motion_len_list = []

for name_code in test_pieces:#midi_path_list:
    # read MIDI
    read_piano_roll = read_midi(UoE_violin_midi_path + "vio01_" + name_code + ".mid")
    midi_len = len(read_piano_roll[0])
    print("seq_len of midi data:", len(read_piano_roll[0]))
    piano_roll_length_list.append(len(read_piano_roll[0]))
    print("midi data shape: ", read_piano_roll.T.shape)

    # read Audio
    read_audio_mfcc = audio_preprocess(UoE_violin_audio_path + "vio01_" + name_code + ".wav", change_fps)
    audio_len = len(read_audio_mfcc)
    print("seq_len of audio data:", len(read_audio_mfcc))
    audio_len_list.append(len(read_audio_mfcc))
    print("audio data shape: ", read_audio_mfcc.shape)
    
    # read Motion
    motion_input = read_motion(UoE_violin_motion_path + "vio01_" + name_code + "_motion_data.pkl")
    motion_len = len(motion_input)
    print("seq_len of motion data:", len(motion_input))
    motion_len_list.append(len(motion_input))
    print("motion data shape: ", motion_input.shape)
    
    cut_len = min(midi_len, audio_len, motion_len)

    piano_roll_dictionary[this_performer][name_code] = read_piano_roll.T[:cut_len]
    audio_dictionary[this_performer][name_code] = read_audio_mfcc[:cut_len]
    motion_dictionary[this_performer][name_code] = motion_input[:cut_len]
    
    print("[cut]midi data shape: ", piano_roll_dictionary[this_performer][name_code].shape)
    print("[cut]audio data shape: ", audio_dictionary[this_performer][name_code].shape)
    print("[cut]motion data shape: ", motion_dictionary[this_performer][name_code].shape)
    
    print("========")
    # read_mfcc_transpose = read_audio_mfcc
    # audio_len = len(read_mfcc_transpose)

# print(piano_roll_dictionary)

seq_len of midi data: 7355
midi data shape:  (7355, 128)


y.shape (8291800,)
sample rate:  44000
mfcc_energy (14, 7539)
mfcc_delta (14, 7539)
mel_scale_sgram (128, 7539)
hop: 1100
aud: (7539, 156)
seq_len of audio data: 7539
audio data shape:  (7539, 156)
seq_len of motion data: 7524
motion data shape:  (7524, 115)
[cut]midi data shape:  (7355, 128)
[cut]audio data shape:  (7355, 156)
[cut]motion data shape:  (7355, 115)
seq_len of midi data: 4618
midi data shape:  (4618, 128)
y.shape (5247880,)
sample rate:  44000
mfcc_energy (14, 4771)
mfcc_delta (14, 4771)
mel_scale_sgram (128, 4771)
hop: 1100
aud: (4771, 156)
seq_len of audio data: 4771
audio data shape:  (4771, 156)
seq_len of motion data: 4695
motion data shape:  (4695, 115)
[cut]midi data shape:  (4618, 128)
[cut]audio data shape:  (4618, 156)
[cut]motion data shape:  (4618, 115)


In [7]:
body_joint_dict = {"RFHD_X":0,"RFHD_Y":1,"RFHD_Z":2, # head
              "LFHD_X":3,"LFHD_Y":4,"LFHD_Z":5,
              "RBHD_X":6,"RBHD_Y":7,"RBHD_Z":8,
              "LBHD_X":9,"LBHD_Y":10,"LBHD_Z":11,
              "C7_X":12,"C7_Y":13,"C7_Z":14,
              "T10_X":15,"T10_Y":16,"T10_Z":17,
              "CLAV_X":18,"CLAV_Y":19,"CLAV_Z":20,
              "STRN_X":21,"STRN_Y":22,"STRN_Z":23,
              "RSHO_X":24,"RSHO_Y":25,"RSHO_Z":26,
              "RELB_X":27,"RELB_Y":28,"RELB_Z":29,
              "RWRA_X":30,"RWRA_Y":31,"RWRA_Z":32,
              "RWRB_X":33,"RWRB_Y":34,"RWRB_Z":35,
              "RFIN_X":36,"RFIN_Y":37,"RFIN_Z":38,
              "LSHO_X":39,"LSHO_Y":40,"LSHO_Z":41,
              "LELB_X":42,"LELB_Y":43,"LELB_Z":44,
              "LWRA_X":45,"LWRA_Y":46,"LWRA_Z":47,
              "LWRB_X":48,"LWRB_Y":49,"LWRB_Z":50,
              "LFIN_X":51,"LFIN_Y":52,"LFIN_Z":53,
              "RASI_X":54,"RASI_Y":55,"RASI_Z":56,#(18*3) left waist
              "LASI_X":57,"LASI_Y":58,"LASI_Z":59,#(19*3) right waist
              "RPSI_X":60,"RPSI_Y":61,"RPSI_Z":62,
              "LPSI_X":63,"LPSI_Y":64,"LPSI_Z":65,
              "RKNE_X":66,"RKNE_Y":67,"RKNE_Z":68,
              "RHEE_X":69,"RHEE_Y":70,"RHEE_Z":71,
              "RTOE_X":72,"RTOE_Y":73,"RTOE_Z":74,
              "RANK_X":75,"RANK_Y":76,"RANK_Z":77,
              "LKNE_X":78,"LKNE_Y":79,"LKNE_Z":80,
              "LHEE_X":81,"LHEE_Y":82,"LHEE_Z":83,
              "LTOE_X":84,"LTOE_Y":85,"LTOE_Z":86,
              "LANK_X":87,"LANK_Y":88,"LANK_Z":89, #29*3
              "Rightant_X":90,"Rightant_Y":91,"Rightant_Z":92,
              "Rightpost_X":93,"Rightpost_Y":94,"Rightpost_Z":95,
              "Leftant_X":96,"Leftant_Y":97,"Leftant_Z":98,
              "Leftpost_X":99,"Leftpost_Y":100,"Leftpost_Z":101}

In [8]:
#Sampling frequency: piano->250, violin->120
sf = 120 #violin
# sf = 250 #piano

In [9]:
def predict(model, input, device):
    model.eval()
    with torch.no_grad():
        input = torch.as_tensor(input).to(torch.float32).to(device)
        # print(target.shape)
        # target = torch.as_tensor(target).to(torch.float32).to(device)
        outputs = model(input)
        return outputs.cpu().numpy()

In [10]:
def column(matrix, i):
    return [row[i] for row in matrix]

def test_render_animation(fps, output, azim, prediction, ground_truth=None):
    prediction_array = np.asarray(prediction)
    print(prediction_array.size)
    limit = len(prediction_array)
    print("limit", limit)
    size = 6#6
    fps = 40

    # Skeleton layout
    parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                [8, 18], [8, 20], [13, 21], [13, 19],
                [5, 20], [5, 21], [7, 18], [7, 19],
                [18, 19], [19, 21], [21, 20], [20, 18], #waist
                [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
    # joints_right = [1, 2, 12, 13, 14]

    prediction_array[:, :, 2] += 0.1 #[:, :, 2]
    if ground_truth is not None:
        ground_truth[:, :, 2] += 0.1
        poses = {'Prediction': prediction_array,
                 'Ground_truth': ground_truth}
    else:
        poses = {'Prediction': prediction_array}
    

    fig = plt.figure()#(figsize=(size*len(poses), size))
    # ax_3d = []
    # lines_3d = []
    radius = 1#14 #3.7#
    # print(poses)
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, len(poses), index + 1, projection='3d')
        ax.clear()
        print(data)
        ims = [] #每一 frame 都存
        for frame_index, each_frame in enumerate(data):
            # print("each_frame")
            # print(each_frame)
            ax.view_init(elev=15., azim=azim)
            ax.set_xlim3d([-radius/2, radius/2])
            ax.set_zlim3d([0, radius])
            ax.set_ylim3d([-radius/2, radius/2])
            ax.set_aspect('auto') #ax.set_aspect('equal')

            # print(title)
            points = ax.scatter(column(each_frame[:30], 0), column(each_frame[:30], 1), column(each_frame[:30], 2), cmap='jet', marker='o', label='body joint', color = 'black')
            points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
            points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
            
            # ax.scatter(column(each_frame, 0), column(each_frame, 1), column(each_frame, 2), cmap='jet', marker='o', label='body joint')
            # ax.legend()
            # print("+++")
            
            parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                        [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                        [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                        [8, 18], [8, 20], [13, 21], [13, 19],
                        [5, 20], [5, 21], [7, 18], [7, 19],
                        [18, 19], [19, 21], [21, 20], [20, 18], #waist
                        [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                        [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                        [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                        [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                        [30, 31], [32, 33],  #instrument
                        # [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
            lines = []
            # draw line
            
            # lines = [ax.plot([each_frame[vs][0], each_frame[ve][0]],
            #                  [each_frame[vs][1], each_frame[ve][1]],
            #                  [each_frame[vs][2], each_frame[ve][2]]) for (vs, ve) in parents]
            line_num = len(parents)
            for idx, each_line in enumerate(parents):
                vec_start = each_frame[each_line[0]]
                vec_end = each_frame[each_line[1]]
                # print(vec_start)
                # print(vec_end)
                line_color = "black"
                if idx == line_num-2:
                    line_color = "blue"
                if idx == line_num-1:
                    line_color = "red"
                # ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]])
                
                temp, = ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]], color=line_color)
                lines.append(temp)

            # ax.figure.savefig('./test_pic/pic' + str(frame_index) + '.png', dpi=100, bbox_inches = 'tight')

            # ims.append([points])
            # image_frame = [points].extend(lines)
            ims.append([points]+[points_2]+[points_3]+lines) #TODO: try extend

            # plt.cla()
            # print("+++")

    anim = matplotlib.animation.ArtistAnimation(fig, ims, interval=1000/fps)

    if output.endswith('.mp4'):
        FFwriter = matplotlib.animation.FFMpegWriter(fps=fps, extra_args=['-vcodec', 'libx264'])
        anim.save(output, writer=FFwriter)
    elif output.endswith('.gif'):
        anim.save(output, fps=fps, dpi=100, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')

In [11]:
def plot(audio_path, plot_path, prediction, sample_time, fps, name=""): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    fluid_syn = FluidSynth()
    fluid_syn.midi_to_audio(audio_path, './output' + name + '.wav')
    input_audio = ffmpeg.input('./output' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [12]:
def plot_wav(audio_path, plot_path, prediction, sample_time, fps, name): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    input_audio = ffmpeg.input(audio_path)
    
    if "Flower" in name:
        isExisting = os.path.exists('./out_tempclip_' + name + '.wav')
        if isExisting != True:
            audio_cut = input_audio.audio.filter('atrim', start=64, end=84)
            audio_output = ffmpeg.output(audio_cut, 'out_tempclip_' + name + '.wav')
            ffmpeg.run(audio_output)
        input_audio = ffmpeg.input('./out_tempclip_' + name + '.wav')
    
    if "Elgar" in name:
        isExisting = os.path.exists('./out_tempclip_' + name + '.wav')
        if isExisting != True:
            audio_cut = input_audio.audio.filter('atrim', start=103, end=123) #, start=30, end=60
            audio_output = ffmpeg.output(audio_cut, './out_tempclip_' + name + '.wav')
            ffmpeg.run(audio_output)
        input_audio = ffmpeg.input('./out_tempclip_' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [13]:
def reset_weights(model): # reset the weight every fold
    if isinstance(model, nn.LSTM) or isinstance(model, nn.Linear):
        model.reset_parameters()

In [14]:
class LSTM1(nn.Module):
    def __init__(self, output_dim, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.output_dim = output_dim #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, output_dim) #fully connected to determine output dim

        self.relu = nn.ReLU()

    def forward(self,x):
        # h0, c0 no time information
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # x is MIDI => [44, 512, 128]

        # hn is final state, run over the sequence length
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # print("output.shape", output.shape)
        # print("hn.shape", hn.shape)
        # out = self.relu(hn)
        out = self.fc_1(output) #final
        return out
 

In [15]:
# Define the model architecture
input_size_midi = 128
input_size_audio = 156#156+128 #number of features
input_size_both = 284

hidden_size = 1024 #1024 #512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 512

output_dim_with_anno = 115 #number of output classes
output_dim_no_anno = 102

batch_size_define_with_anno = 20#128
batch_size_define_no_anno = 128

learning_rate = 0.001#0.001

In [52]:
def model_predict(model_path, input_size, output_dim, test_data, filecode, loss_type):
    model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(checkpoint['model_state_dict'])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.eval()
    
    data_format = ""
    if input_size == 128:
        data_format = "MIDI"
    if input_size == 156:
        data_format = "Audio"
    if input_size == 284:
        data_format = "Both"
    
    NoY = ""
    if output_dim == 102:
        NoY = "no"
    if output_dim == 115:
        NoY = "yes"
    
    num_epochs = 0
    if loss_type == "mse":
        num_epochs = 500
    if loss_type == "new":
        num_epochs = 300
    
    full_prediction = pd.DataFrame()
    num_count = 0
    # read midi
    # test_dataloader = get_dataloader(test_datapath, batch_size=1)
    # for test_batch in test_data_list:
    with torch.no_grad():
        # first_target = torch.zeros(test_batch.shape[0],112)
        # print(first_target.shape)
        test_input = test_data[None, :]
        # test_target = first_target[None, :]
        print("test_input", test_input.shape)
        # print("test_target", test_target.shape)
        prediction = predict(model, test_input, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)

        # print(len(Row_list_prediction), len(Row_list_prediction[0]),len(Row_list_prediction[0][0]))
        # UoE_violin_midi_path = "./UoE_violin_midi/violin_midi/vio01/"
        # if data_format == "MIDI":
        #     # MIDI input
        #     plot(UoE_violin_midi_path + "vio01_" + filecode + ".mid", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # else:
        # wav input
        
        if "Elgar" in filecode:#4120
            plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[4120:4920], None, 40, filecode) #ow_list[0:900]

        if "Flower" in filecode:#2560
            plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[2560:3360], None, 40, filecode) #ow_list[0:900]
        
        else:
            plot_wav("output" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # print("prediction.shape", prediction.shape)
        prediction_arr = np.array(Row_list_prediction)
        if not os.path.exists("./save_prediction/[" + data_format + "_" + NoY + "_anno](" + loss_type + ")"+str(num_layers)+"LSTM_hidden"+str(hidden_size)+'_'+str(num_epochs)+"epoch/"):
            os.makedirs("./save_prediction/[" + data_format + "_" + NoY + "_anno](" + loss_type + ")"+str(num_layers)+"LSTM_hidden"+str(hidden_size)+'_'+str(num_epochs)+"epoch/")
        both_data_output = open("./save_prediction/[" + data_format + "_" + NoY + "_anno](" + loss_type + ")"+str(num_layers)+"LSTM_hidden"+str(hidden_size)+'_'+str(num_epochs)+"epoch/prediction_"+
                                filecode +".pkl", 'wb')
        pickle.dump(prediction_arr, both_data_output)
        both_data_output.close()
        # formated_motion = prediction_format(full_prediction)
        # # # plot(formated_motion)
        # audio_path = test_music_list[num_count][0]
        # output_path = "test_output_" + filecode + ".mp4"
        # plot(formated_motion, audio_path, output_path, None, 10, filecode)
        num_count += 1

    # model.train()

In [17]:
no_anno_model_path = "./no_anno_model_save/"
with_anno_model_path = "./with_anno_model_save/"

no_anno_model_path_list = glob.glob(no_anno_model_path + "*.tar")


for eval_model_path in no_anno_model_path_list:
    for filecode in test_pieces:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        print(eval_model_path)
        input_size = 0
        if ('midi' in eval_model_path):
            input_size = 128
            test_data = piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            test_data = audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            test_data = np.concatenate(
            (piano_roll_dictionary[this_performer][filecode], audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        if ('no_anno' in eval_model_path):
            output_dim = 102
        if ('with_anno' in eval_model_path):
            output_dim = 115
        
        loss_type = ""
        if ("total500" in eval_model_path):
            loss_type = "mse"
        if ("total300" in eval_model_path):
            loss_type = "new"
        print(loss_type)

        model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)

./no_anno_model_save/[midi_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_06-18-09.tar
new


OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.3526, -0.4451,  0.2492,  ...,  0.2224,  0.2192, -0.4368],
        [ 0.0503,  0.2439,  0.2582,  ...,  0.0911, -0.1098,  0.4041],
        [ 0.1968,  0.4012,  0.1053,  ..., -0.3714, -0.3238,  0.0288],
        ...,
        [ 0.1233,  0.4862,  0.0470,  ..., -0.4186, -0.3679,  0.3941],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[midi_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_06-18-09.tar
new
OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.3526, -0.4451,  0.2492,  ...,  0.2224,  0.2192, -0.4368],
        [ 0.0503,  0.2439,  0.2582,  ...,  0.0911, -0.1098,  0.4041],
        [ 0.1968,  0.4012,  0.1053,  ..., -0.3714, -

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[both_no_anno][total500_hs1024]LSTM_save_epoch_499_2023-08-31_23-49-14.tar
mse
OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.0071, -0.0306, -0.0063,  ..., -0.0289, -0.0148,  0.0008],
        [-0.0133, -0.0255,  0.0066,  ...,  0.0146, -0.0054, -0.0036],
        [ 0.0068, -0.0163, -0.0241,  ...,  0.0285,  0.0285, -0.0234],
        ...,
        [ 0.0061, -0.0160, -0.0183,  ...,  0.0085, -0.0006,  0.0278],
        [-0.0273, -0.0243,  0.0005,  ...,  0.0230, -0.0195, -0.0244],
        [ 0.0237, -0.0077, -0.0216,  ..., -0.0057,  0.0007,  0.0242]],
       device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.0213, -0.0311, -0.0203,  ..., -0.0272,  0.0023, -0.0081],
        [ 0.0014,  0.7318, -0.2557,  ...,  0.0145,  0.0804,  0.0914],
        [-0.0199,  0.0136, -0.0307,  ...,  0.0106, -0.0107, -0.0210],
        ...,
        [ 0.0074, -0.0153,  0.0263,  ..., -0.0038, -0.0108, -0.0309],
        [ 0.0287,  0.2851, -0.2392,  ..., -0.0143, -0.1468, -0.1204],
        [-0.0121,  0

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[both_no_anno][total500_hs1024]LSTM_save_epoch_499_2023-08-31_23-49-14.tar
mse
OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.0071, -0.0306, -0.0063,  ..., -0.0289, -0.0148,  0.0008],
        [-0.0133, -0.0255,  0.0066,  ...,  0.0146, -0.0054, -0.0036],
        [ 0.0068, -0.0163, -0.0241,  ...,  0.0285,  0.0285, -0.0234],
        ...,
        [ 0.0061, -0.0160, -0.0183,  ...,  0.0085, -0.0006,  0.0278],
        [-0.0273, -0.0243,  0.0005,  ...,  0.0230, -0.0195, -0.0244],
        [ 0.0237, -0.0077, -0.0216,  ..., -0.0057,  0.0007,  0.0242]],
       device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.0213, -0.0311, -0.0203,  ..., -0.0272,  0.0023, -0.0081],
        [ 0.0014,  0.7318, -0.2557,  ...,  0.0145,  0.0804,  0.0914],
        [-0.0199,  0.0136, -0.0307,  ...,  0.0106, -0.0107, -0.0210],
        ...,
        [ 0.0074, -0.0153,  0.0263,  ..., -0.0038, -0.0108, -0.0309],
        [ 0.0287,  0.2851, -0.2392,  ..., -0.0143, -0.1468, -0.1204],
        [-0.0121,  0

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

In [18]:
with_anno_model_path_list = glob.glob(with_anno_model_path + "*.tar")

for eval_model_path in with_anno_model_path_list:
    for filecode in test_pieces:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        input_size = 0
        if ('midi' in eval_model_path):
            input_size = 128
            test_data = piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            test_data = audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            test_data = np.concatenate(
            (piano_roll_dictionary[this_performer][filecode], audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        if ('no_anno' in eval_model_path):
            output_dim = 102
        if ('with_anno' in eval_model_path):
            output_dim = 115
        
        loss_type = ""
        if ('total500' in eval_model_path):
            loss_type = "mse"
        if ('total300' in eval_model_path):
            loss_type = "new"
            
        model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [-2.2005e-03, -1.4138e-02,  2.1567e-02,  ...,  2.6931e-02,
         -1.0656e-02, -1.4090e-04],
        [ 2.9328e-02, -1.1218e-02, -1.4724e-02,  ...,  2.3250e-02,
         -1.9489e-02,  3.0060e-02],
        [ 1.2342e-02, -2.9460e-02, -1.1747e-02,  ..., -1.9196e-03,
         -6.8986e-03,  1.2311e-02]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.1561, -0.0902,  0.1600,  ..., -0.6004,  0.2462,  0.2367],
        [-0.1749, -0.0397, -0.1754,  ..., -0.5262, -0.1405,  0.0111],
        [ 0.0782, -0.5275, -0.3311,  ..., -0.2035,  0.1834, -0.1874],
        ...,
        [ 0.3344,  0.0308, -0.1157,  ...,  0.4482,  0.1918,  0.1961],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [-2.2005e-03, -1.4138e-02,  2.1567e-02,  ...,  2.6931e-02,
         -1.0656e-02, -1.4090e-04],
        [ 2.9328e-02, -1.1218e-02, -1.4724e-02,  ...,  2.3250e-02,
         -1.9489e-02,  3.0060e-02],
        [ 1.2342e-02, -2.9460e-02, -1.1747e-02,  ..., -1.9196e-03,
         -6.8986e-03,  1.2311e-02]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.1561, -0.0902,  0.1600,  ..., -0.6004,  0.2462,  0.2367],
        [-0.1749, -0.0397, -0.1754,  ..., -0.5262, -0.1405,  0.0111],
        [ 0.0782, -0.5275, -0.3311,  ..., -0.2035,  0.1834, -0.1874],
        ...,
        [ 0.3344,  0.0308, -0.1157,  ...,  0.4482,  0.1918,  0.1961],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.0450, -0.1102,  0.0738,  ..., -0.0476, -0.0084,  0.0192],
        [ 0.0613, -0.0630,  0.0141,  ..., -0.0312,  0.0207,  0.0222],
        [-0.0369, -0.0211, -0.0075,  ...,  0.0089, -0.0174,  0.0293],
        ...,
        [-0.0056,  0.0236, -0.0157,  ...,  0.0160, -0.0132,  0.0161],
        [ 0.0166,  0.0203,  0.0770,  ...,  0.0264,  0.0159,  0.0274],
        [ 0.0557, -0.0239, -0.0433,  ..., -0.0390, -0.0032, -0.0205]],
       device='cuda:0')), ('lstm.weight_hh_l0', tensor([[-1.6452e-01, -2.3143e-02, -3.1819e-02,  ..., -6.2853e-02,
         -6.0886e-03, -8.6548e-03],
        [ 1.9855e-03,  2.6501e-02,  2.2403e-02,  ...,  1.4540e-02,
         -3.2325e-02,  2.0608e-02],
        [-1.9930e-02,  1.3712e-02, -2.9895e-02,  ...,  1.0573e-02,
         -1.8141e-02, -1.7755e-02],
        ...,
        [ 6.1337e-03, -2.5778e-02, -1.6967e-02,  ...,  3.0345e-02,
          2.8063e-02, -2.2014e-02],
        [-8.8435e-03,  2.5552e-02,  1.2934e-02,  ..., -4.1

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[-0.1427,  0.0793, -0.0038,  ..., -0.0929,  0.2412,  0.1976],
        [ 0.0974, -0.0445,  0.1186,  ..., -0.1134,  0.1481, -0.2700],
        [-0.0390,  0.3424,  0.0186,  ...,  0.1923,  0.3449, -0.1556],
        ...,
        [-0.0050, -0.1832,  0.0765,  ..., -0.0732, -0.0953, -0.1902],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[-0.1427,  0.0793, -0.0038,  ..., -0.0929,  0.2412,  0.1976],
        [ 0.0974, -0.0445,  0.1186,  ..., -0.1134,  0.1481, -0.2700],
        [-0.0390,  0.3424,  0.0186,  ...,  0.1923,  0.3449, -0.1556],
        ...,
        [-0.0050, -0.1832,  0.0765,  ..., -0.0732, -0.0953, -0.1902],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

In [19]:
for filecode in test_pieces:
    ground_truth = pd.DataFrame(motion_dictionary[this_performer][filecode][:, :102])
    print(ground_truth.shape)
    
    Row_list_prediction =[]
    
    # Iterate over each row
    for index, rows in ground_truth.iterrows():
        #fill nan
        rows = rows.fillna(0)
        # Create list for the current row
        my_list = rows.values.tolist()
        # print(my_list)
        
        my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
        # append the list to the final list
        Row_list_prediction.append(my_list_per3)

    print(len(Row_list_prediction))
    
    if "Elgar" in filecode:#4120
        plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[ground_truth_40fps]video_" + filecode + ".mp4", Row_list_prediction[4120:4920], None, 40, filecode)
        # plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[4120:4920], None, 40, filecode) #ow_list[0:900]

    if "Flower" in filecode:#2560
        plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[ground_truth_40fps]video_" + filecode + ".mp4", Row_list_prediction[2560:3360], None, 40, filecode)
        # plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[2560:3360], None, 40, filecode) #ow_list[0:900]


(7355, 102)
7355
81600
limit 800
[[[-0.03610143  0.15411506  1.1       ]
  [-0.10079484  0.11515813  1.09286173]
  [ 0.00812141  0.08827308  1.12957553]
  ...
  [-0.05111423  0.11144011  0.9691206 ]
  [-0.27138346  0.07299885  1.1497737 ]
  [-0.01497577  0.1949214   0.81660917]]

 [[-0.03370006  0.15315776  1.1       ]
  [-0.09846268  0.11443976  1.0929706 ]
  [ 0.01043738  0.08702317  1.12891869]
  ...
  [-0.04918113  0.11131305  0.9688737 ]
  [-0.26967146  0.07284576  1.14708432]
  [-0.01162964  0.19624466  0.8162821 ]]

 [[-0.03121112  0.15211944  1.1       ]
  [-0.09608648  0.11371011  1.09309974]
  [ 0.01279568  0.08571753  1.1282602 ]
  ...
  [-0.04723838  0.11120911  0.96870062]
  [-0.26762478  0.07283121  1.14460294]
  [-0.00821867  0.1973379   0.81579855]]

 ...

 [[ 0.01047852  0.10458845  1.1       ]
  [-0.055626    0.07094083  1.10043272]
  [ 0.05384181  0.03543664  1.10427997]
  ...
  [-0.02839767  0.09332386  0.97165568]
  [-0.12788888  0.13848323  1.0541324 ]
  [ 0.06442

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

(4618, 102)
4618
81600
limit 800
[[[ 0.0076429   0.10942956  1.1       ]
  [-0.06106092  0.08150571  1.09799737]
  [ 0.04437959  0.03773931  1.11557397]
  ...
  [-0.02567353  0.08603249  0.96666925]
  [-0.22633986  0.08540441  1.14550308]
  [ 0.01221213  0.18240152  0.7999958 ]]

 [[ 0.00898266  0.10819265  1.1       ]
  [-0.05976609  0.08053631  1.09813411]
  [ 0.04553568  0.03634767  1.11489449]
  ...
  [-0.02476687  0.0866778   0.96679868]
  [-0.22341991  0.0863482   1.14231684]
  [ 0.0159237   0.18389051  0.79809945]]

 [[ 0.01036273  0.10681734  1.1       ]
  [-0.05842996  0.07941319  1.09828237]
  [ 0.04678437  0.034895    1.11419198]
  ...
  [-0.02403419  0.08645453  0.96687993]
  [-0.22027473  0.08692967  1.13875618]
  [ 0.01984899  0.18482941  0.79581554]]

 ...

 [[ 0.03196638  0.11817119  1.1       ]
  [-0.03589529  0.08891034  1.10568288]
  [ 0.07146871  0.04722506  1.11078013]
  ...
  [-0.00904666  0.09604553  0.96901543]
  [-0.18867901  0.11597971  1.16140951]
  [ 0.04055

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

In [22]:
UoE_violin_midi_path = "./UoE_violin_midi/violin_midi/vio01/"
UoE_violin_audio_path = "./UoE_violin_midi/performance_audio/vio01/"
UoE_violin_motion_path = "./preprocessed_data_save_cross/motion/"
#midi_path_list = glob.glob(UoE_violin_midi_path + "/" + this_performer + "/*.mid")

eval_piano_roll_dictionary = {}
eval_piano_roll_dictionary[this_performer] = {}
eval_piano_roll_length_list = []

eval_audio_dictionary = {}
eval_audio_dictionary[this_performer] = {}
eval_audio_len_list = []

eval_motion_dictionary = {}
eval_motion_dictionary[this_performer] = {}
eval_motion_len_list = []

for name_code in test_pieces_eval:#midi_path_list:
    # read MIDI
    read_piano_roll = read_midi(UoE_violin_midi_path + "vio01_" + name_code + ".mid")
    midi_len = len(read_piano_roll[0])
    print("seq_len of midi data:", len(read_piano_roll[0]))
    eval_piano_roll_length_list.append(len(read_piano_roll[0]))
    print("midi data shape: ", read_piano_roll.T.shape)

    # read Audio
    read_audio_mfcc = audio_preprocess(UoE_violin_audio_path + "vio01_" + name_code + ".wav", change_fps)
    audio_len = len(read_audio_mfcc)
    print("seq_len of audio data:", len(read_audio_mfcc))
    eval_audio_len_list.append(len(read_audio_mfcc))
    print("audio data shape: ", read_audio_mfcc.shape)
    
    # read Motion
    motion_input = read_motion(UoE_violin_motion_path + "vio01_" + name_code + "_motion_data.pkl")
    motion_len = len(motion_input)
    print("seq_len of motion data:", len(motion_input))
    eval_motion_len_list.append(len(motion_input))
    print("motion data shape: ", motion_input.shape)
    
    cut_len = min(midi_len, audio_len, motion_len)

    eval_piano_roll_dictionary[this_performer][name_code] = read_piano_roll.T[:cut_len]
    eval_audio_dictionary[this_performer][name_code] = read_audio_mfcc[:cut_len]
    eval_motion_dictionary[this_performer][name_code] = motion_input[:cut_len]
    
    print("[cut]midi data shape: ", eval_piano_roll_dictionary[this_performer][name_code].shape)
    print("[cut]audio data shape: ", eval_audio_dictionary[this_performer][name_code].shape)
    print("[cut]motion data shape: ", eval_motion_dictionary[this_performer][name_code].shape)
    
    print("========")
    # read_mfcc_transpose = read_audio_mfcc
    # audio_len = len(read_mfcc_transpose)

# print(piano_roll_dictionary)

seq_len of midi data: 7355
midi data shape:  (7355, 128)
y.shape (8291800,)
sample rate:  44000
mfcc_energy (14, 7539)
mfcc_delta (14, 7539)
mel_scale_sgram (128, 7539)
hop: 1100
aud: (7539, 156)
seq_len of audio data: 7539
audio data shape:  (7539, 156)
seq_len of motion data: 7524
motion data shape:  (7524, 115)
[cut]midi data shape:  (7355, 128)
[cut]audio data shape:  (7355, 156)
[cut]motion data shape:  (7355, 115)
seq_len of midi data: 7785
midi data shape:  (7785, 128)
y.shape (8769201,)
sample rate:  44000
mfcc_energy (14, 7973)
mfcc_delta (14, 7973)
mel_scale_sgram (128, 7973)
hop: 1100
aud: (7973, 156)
seq_len of audio data: 7973
audio data shape:  (7973, 156)
seq_len of motion data: 7955
motion data shape:  (7955, 115)
[cut]midi data shape:  (7785, 128)
[cut]audio data shape:  (7785, 156)
[cut]motion data shape:  (7785, 115)
seq_len of midi data: 4618
midi data shape:  (4618, 128)
y.shape (5247880,)
sample rate:  44000
mfcc_energy (14, 4771)
mfcc_delta (14, 4771)
mel_scale_s

In [32]:
device

device(type='cuda', index=0)

In [42]:
test_data_list = ["Elgar_S1_T1", "Elgar_S1_T2", "Flower_S1_T1", "Flower_S1_T2", "Mend_S1_T2", "Mozart1_S1_T1", "Mozart1_S1_T2", "Mozart2_S1_T1", "Mozart2_S1_T2"]

for eval_model_path in with_anno_model_path_list:
    outputs_save = []
    counter = 0
    valid_running_loss = 0.0
    for filecode in test_pieces_eval:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        input_size = 0
        data_format = ""
        if ('midi' in eval_model_path):
            input_size = 128
            data_format = "midi"
            test_data = eval_piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            data_format = "audio"
            test_data = eval_audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            data_format = "both"
            test_data = np.concatenate(
            (eval_piano_roll_dictionary[this_performer][filecode], eval_audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        YoN = ""
        if ('no_anno' in eval_model_path):
            output_dim = 102
            YoN = "no"
        if ('with_anno' in eval_model_path):
            output_dim = 115
            YoN = "with"
        
        num_epochs = 0
        loss_type = ""
        if ('total500' in eval_model_path):
            loss_type = "mse"
            num_epochs = 500
        if ('total300' in eval_model_path):
            loss_type = "new"
            num_epochs = 300
            
        # model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)
        model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
        checkpoint = torch.load(eval_model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(checkpoint['model_state_dict'])
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        ground_truth_data = eval_motion_dictionary[this_performer][filecode]
        # evaluate_lstm_cross(model, num_epochs)
        with torch.no_grad():
            counter += 1

            # inputs = test_data.to(device).float()
            # targets = targets.to(device).float()
            # inputs = np.array(test_data[None, :], dtype=float).to(device)
            # targets = np.array(ground_truth_data[None, :], dtype=float).to(device)
            inputs = torch.from_numpy(test_data[None, :]).float().to(device)
            targets = torch.from_numpy(ground_truth_data[None, :]).float().to(device)
            print("val inputs.shape:", inputs.shape)
            print("val targets.shape:", targets.shape)
            outputs = model(inputs)
            print("val outputs.shape:", outputs.shape)

            loss =  F.mse_loss(outputs, targets)
            valid_running_loss += loss.cpu().item()
            # previous_output = outputs
            outputs_save.append(np.asarray(outputs[0].cpu()))

        loc_dt = datetime.datetime.today()
        loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
        if not os.path.exists("./output_eval/"):
            os.makedirs('./output_eval/')

    print(len(outputs_save), outputs_save[0].shape)
    eval_output = open("./output_eval/[new][" + data_format + "_" + YoN + "_anno][total" + str(num_epochs) + "_hs" + str(hidden_size) +"]save_"+ str(loc_dt_format) + "_l1_loss_" + str(loss.cpu().item())+".pkl", 'wb')
    pickle.dump(np.asarray(outputs_save), eval_output)
    eval_output.close()

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [-2.2005e-03, -1.4138e-02,  2.1567e-02,  ...,  2.6931e-02,
         -1.0656e-02, -1.4090e-04],
        [ 2.9328e-02, -1.1218e-02, -1.4724e-02,  ...,  2.3250e-02,
         -1.9489e-02,  3.0060e-02],
        [ 1.2342e-02, -2.9460e-02, -1.1747e-02,  ..., -1.9196e-03,
         -6.8986e-03,  1.2311e-02]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.1561, -0.0902,  0.1600,  ..., -0.6004,  0.2462,  0.2367],
        [-0.1749, -0.0397, -0.1754,  ..., -0.5262, -0.1405,  0.0111],
        [ 0.0782, -0.5275, -0.3311,  ..., -0.2035,  0.1834, -0.1874],
        ...,
        [ 0.3344,  0.0308, -0.1157,  ...,  0.4482,  0.1918,  0.1961],

In [45]:
for eval_model_path in no_anno_model_path_list:
    outputs_save = []
    counter = 0
    valid_running_loss = 0.0
    for filecode in test_pieces_eval:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        input_size = 0
        data_format = ""
        if ('midi' in eval_model_path):
            input_size = 128
            data_format = "midi"
            test_data = eval_piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            data_format = "audio"
            test_data = eval_audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            data_format = "both"
            test_data = np.concatenate(
            (eval_piano_roll_dictionary[this_performer][filecode], eval_audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        YoN = ""
        if ('no_anno' in eval_model_path):
            output_dim = 102
            YoN = "no"
        if ('with_anno' in eval_model_path):
            output_dim = 115
            YoN = "with"
        
        num_epochs = 0
        loss_type = ""
        if ('total500' in eval_model_path):
            loss_type = "mse"
            num_epochs = 500
        if ('total300' in eval_model_path):
            loss_type = "new"
            num_epochs = 300
            
        # model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)
        model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
        checkpoint = torch.load(eval_model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(checkpoint['model_state_dict'])
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        ground_truth_data = eval_motion_dictionary[this_performer][filecode]
        # evaluate_lstm_cross(model, num_epochs)
        with torch.no_grad():
            counter += 1

            # inputs = test_data.to(device).float()
            # targets = targets.to(device).float()
            # inputs = np.array(test_data[None, :], dtype=float).to(device)
            # targets = np.array(ground_truth_data[None, :], dtype=float).to(device)
            inputs = torch.from_numpy(test_data[None, :]).float().to(device)
            targets = torch.from_numpy(ground_truth_data[None, :, :102]).float().to(device)
            print("val inputs.shape:", inputs.shape)
            print("val targets.shape:", targets.shape)
            outputs = model(inputs)
            print("val outputs.shape:", outputs.shape)

            loss =  F.mse_loss(outputs, targets)
            valid_running_loss += loss.cpu().item()
            # previous_output = outputs
            outputs_save.append(np.asarray(outputs[0].cpu()))

        loc_dt = datetime.datetime.today()
        loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
        if not os.path.exists("./output_eval/"):
            os.makedirs('./output_eval/')

    print(len(outputs_save), outputs_save[0].shape)
    eval_output = open("./output_eval/[new][" + data_format + "_" + YoN + "_anno][total" + str(num_epochs) + "_hs" + str(hidden_size) +"]save_"+ str(loc_dt_format) + "_l1_loss_" + str(loss.cpu().item())+".pkl", 'wb')
    pickle.dump(np.asarray(outputs_save), eval_output)
    eval_output.close()

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.3526, -0.4451,  0.2492,  ...,  0.2224,  0.2192, -0.4368],
        [ 0.0503,  0.2439,  0.2582,  ...,  0.0911, -0.1098,  0.4041],
        [ 0.1968,  0.4012,  0.1053,  ..., -0.3714, -0.3238,  0.0288],
        ...,
        [ 0.1233,  0.4862,  0.0470,  ..., -0.4186, -0.3679,  0.3941],

In [48]:
test_midi_path = "./BWV1001/"
test_audio_path = "./BWV1001/"

test_piano_roll_dictionary = {}
test_piano_roll_dictionary[this_performer] = {}
test_piano_roll_length_list = []

test_audio_dictionary = {}
test_audio_dictionary[this_performer] = {}
test_audio_len_list = []


for name_code in test_pieces_test:#midi_path_list:
    # read MIDI
    read_piano_roll = read_midi(test_midi_path + name_code + ".mid")
    midi_len = len(read_piano_roll[0])
    print("seq_len of midi data:", len(read_piano_roll[0]))
    test_piano_roll_length_list.append(len(read_piano_roll[0]))
    print("midi data shape: ", read_piano_roll.T.shape)

    # read Audio
    read_audio_mfcc = audio_preprocess(test_audio_path + name_code + ".wav", change_fps)
    audio_len = len(read_audio_mfcc)
    print("seq_len of audio data:", len(read_audio_mfcc))
    test_audio_len_list.append(len(read_audio_mfcc))
    print("audio data shape: ", read_audio_mfcc.shape)

    cut_len = min(midi_len, audio_len)

    test_piano_roll_dictionary[this_performer][name_code] = read_piano_roll.T[:cut_len]
    test_audio_dictionary[this_performer][name_code] = read_audio_mfcc[:cut_len]
    
    print("[cut] test midi data shape: ", test_piano_roll_dictionary[this_performer][name_code].shape)
    print("[cut] test audio data shape: ", test_audio_dictionary[this_performer][name_code].shape)
    
    print("========")
    # read_mfcc_transpose = read_audio_mfcc
    # audio_len = len(read_mfcc_transpose)

# print(piano_roll_dictionary)

seq_len of midi data: 8171
midi data shape:  (8171, 128)
y.shape (8988979,)
sample rate:  44000
mfcc_energy (14, 8172)
mfcc_delta (14, 8172)
mel_scale_sgram (128, 8172)
hop: 1100
aud: (8172, 156)
seq_len of audio data: 8172
audio data shape:  (8172, 156)
[cut] test midi data shape:  (8171, 128)
[cut] test audio data shape:  (8171, 156)


In [53]:

for eval_model_path in no_anno_model_path_list:
    for filecode in test_pieces_test:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        print(eval_model_path)
        input_size = 0
        if ('midi' in eval_model_path):
            input_size = 128
            test_data = test_piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            test_data = test_audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            test_data = np.concatenate(
            (test_piano_roll_dictionary[this_performer][filecode], test_audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        if ('no_anno' in eval_model_path):
            output_dim = 102
        if ('with_anno' in eval_model_path):
            output_dim = 115
        
        loss_type = ""
        if ("total500" in eval_model_path):
            loss_type = "mse"
        if ("total300" in eval_model_path):
            loss_type = "new"
        print(loss_type)

        model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)

./no_anno_model_save/[midi_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_06-18-09.tar
new
OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.3526, -0.4451,  0.2492,  ...,  0.2224,  0.2192, -0.4368],
        [ 0.0503,  0.2439,  0.2582,  ...,  0.0911, -0.1098,  0.4041],
        [ 0.1968,  0.4012,  0.1053,  ..., -0.3714, -

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02.tar
new
OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.0353, -0.0612,  0.0111,  ..., -0.0255, -0.0007,  0.0204],
        [ 0.0522, -0.0416,  0.0589,  ..., -0.0331,  0.0200,  0.0221],
        [-0.0005, -0.1292,  0.0113,  ...,  0.0150, -0.0161,  0.0294],
        ...,
        [-0.0496,  0.0679, -0.0745,  ...,  0.0183, -0.0207, -0.0290],
        [-0.0252, -0.0324, -0.0099,  ..., -0.0012,  0.0061, -0.0144],
        [ 0.0376, -0.0422, -0.0062,  ..., -0.0047, -0.0203,  0.0212]],
       device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.0212, -0.0191,  0.0104,  ...,  0.0119,  0.0306,  0.0137],
        [ 0.0109, -0.0492, -0.0293,  ..., -0.0548, -0.0102, -0.0294],
        [-0.0266,  0.0213,  1.1847,  ...,  0.2372,  0.0276, -0.0149],
        ...,
        [ 0.0101,  0.0271,  0.0048,  ...,  0.0026, -0.0093,  0.0095],
        [ 0.0018, -0.0101,  0.0284,  ..., -0.0292, -0.0101,  0.0029],
        [-0.0144, -

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[audio_no_anno][total500_hs1024]LSTM_save_epoch_499_2023-08-31_16-28-44.tar
mse
OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.0181, -0.0548, -0.0054,  ..., -0.0255, -0.0007,  0.0204],
        [ 0.0480, -0.0460,  0.0648,  ..., -0.0329,  0.0200,  0.0221],
        [ 0.0234, -0.0996,  0.0230,  ...,  0.0086, -0.0175,  0.0293],
        ...,
        [-0.0147,  0.0344, -0.0127,  ...,  0.0165, -0.0218, -0.0290],
        [-0.0364,  0.0734, -0.0073,  ..., -0.0021,  0.0059, -0.0145],
        [-0.0017, -0.0191, -0.0153,  ..., -0.0047, -0.0203,  0.0212]],
       device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.0205, -0.0195,  0.0157,  ...,  0.0169,  0.0322,  0.0137],
        [ 0.0147, -0.0250, -0.0279,  ..., -0.0583,  0.0347, -0.0294],
        [-0.0258,  0.0131, -0.0296,  ..., -0.0076,  0.0342, -0.0149],
        ...,
        [ 0.0117,  0.0181,  0.0071,  ..., -0.0044, -0.5246,  0.0096],
        [ 0.0018, -0.0103,  0.0285,  ..., -0.0376, -0.6096,  0.0029],
        [-0.0144, -

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[midi_no_anno][total500_hs1024]LSTM_save_epoch_499_2023-09-05_20-26-03_avg_loss_0.00015126124086651266.tar
mse
OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.0124,  0.0764,  0.1066,  ...,  0.0527, -0.0196, -0.0937],
        [-0.0537, -0.1022, -0.0849,  ...,  0.0729,  0.0067,  0.0239],
        [ 0.1115,  0

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[both_no_anno][total500_hs1024]LSTM_save_epoch_499_2023-08-31_23-49-14.tar
mse
OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.0071, -0.0306, -0.0063,  ..., -0.0289, -0.0148,  0.0008],
        [-0.0133, -0.0255,  0.0066,  ...,  0.0146, -0.0054, -0.0036],
        [ 0.0068, -0.0163, -0.0241,  ...,  0.0285,  0.0285, -0.0234],
        ...,
        [ 0.0061, -0.0160, -0.0183,  ...,  0.0085, -0.0006,  0.0278],
        [-0.0273, -0.0243,  0.0005,  ...,  0.0230, -0.0195, -0.0244],
        [ 0.0237, -0.0077, -0.0216,  ..., -0.0057,  0.0007,  0.0242]],
       device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.0213, -0.0311, -0.0203,  ..., -0.0272,  0.0023, -0.0081],
        [ 0.0014,  0.7318, -0.2557,  ...,  0.0145,  0.0804,  0.0914],
        [-0.0199,  0.0136, -0.0307,  ...,  0.0106, -0.0107, -0.0210],
        ...,
        [ 0.0074, -0.0153,  0.0263,  ..., -0.0038, -0.0108, -0.0309],
        [ 0.0287,  0.2851, -0.2392,  ..., -0.0143, -0.1468, -0.1204],
        [-0.0121,  0

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55